# Simple finite volume solver

## Prerequisites (libraries and helper functions)
We start loading several libraries:

In [ ]:
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry
ngsglobals.msg_level = 1
from ngsolve.meshes import *
#from draw import Draw


We may need function for absolute value and maxima computations on `CoefficientFunctions` which we compose with the `IfPos` function:

In [ ]:
def Abs(u):
    return IfPos(u,u,-u)

def Max(u,v):
    return IfPos(u-v,u,v)

For drawing 1D Plots on cross sections of the mesh we introduce a helper function that assumes a regular mesh with $N$ elements in x-direction:

In [ ]:
def DrawOnCrossSection(gfu,N):    
    import matplotlib.pyplot as plt
    from numpy import nan
    eps = 1e-6
    x_s=[]
    y_s=[]
    mesh = gfu.space.mesh
    for i in range(N):
        x_s.append(i/N)
        y_s.append(gfu(mesh(i/N+eps,0.5)))
        x_s.append((i+1)/N)
        y_s.append(gfu(mesh((i+1)/N-eps,0.5)))
        x_s.append(nan)
        y_s.append(nan)
    plt.plot(x_s,y_s,label="u")
    plt.show()

Here is an example of a corresponding mesh and a piecewise constant function defined on that mesh:

In [ ]:
mesh = MakeStructured2DMesh(nx=10,ny=1,quads=True,periodic_x=True,periodic_y=True)
PC = L2(mesh,order=0)
gfdrawtest = GridFunction(PC)
gfdrawtest.Set(sin(2*pi*x))
DrawOnCrossSection(gfdrawtest,10)

## A simple finite volume solver

We can now define a very simple finite volume solver based on:
 * a flux function $F$,
 * a numerical flux function $\hat{f}_n$,
 * initial values $u_0$,
 * boundary data (if needed),
 * a mesh and
 * a time step for an explicit Euler discretization.
 
Let $u_T$ be the constant corresponding to one element. Then the scheme reads as

$$
u_T^{n+1} = u_{T}^n - \frac{\Delta t}{|T|} \sum_{K \in \partial T} 
\hat{f}_{K,n}(u_T,u_{T'},n_K) |K|
$$
    

This, together with some plotting at $t = 0,0.1,..,0.5$ is done in the following function:

In [ ]:
def Solve(F, fhatn, u0, ubnd, mesh, dt):
    V = L2(mesh,order=0)
    gfu=GridFunction(V)
    u,v = V.TnT()
    # definition of the "bilinear" form
    # Note: It -- despite its name -- does not need to be linear in the first argument!
    a = BilinearForm (V, nonassemble=True)
    a += fhatn(u,u.Other(ubnd),specialcf.normal(mesh.dim)) * v * dx(element_boundary=True)
    Draw(gfu,mesh,"u",sd=0)
    t=0

    gfu.Set(u0)
    intu0 = Integrate(gfu,mesh,order=0)
    DrawOnCrossSection(gfu,mesh.ne)
    
    Ts = [0.1,0.2,0.3,0.4,0.5]
    for T in Ts:
        while t < T-dt/2:
            gfu.vec.data -= dt * V.InvM() @ a.mat * gfu.vec
            t += dt
            Redraw()
        print("t = ", t, "conservation error: ", abs(Integrate(gfu,mesh,order=0)-intu0))
        DrawOnCrossSection(gfu,mesh.ne)

## The problem
As the model problem here we consider a trivial extension of the Burgers equation to 2D (where only the flux in x-direction is non-trivial):
$$
F(u(x,y,t)) = (\frac12 u^2, 0)^T
$$

In [ ]:
def F(u):
    return CoefficientFunction((0.5*u**2,0))

The domain will be $\Omega = [0,1]^2$ with periodic boundary conditions everywhere. For the purpose of having the feature available for later we also give the code to provide boundary conditions here:

In [ ]:
ubnd_dir = {"bottom" : 0, "right" : 0, "top" : 0, "left" : 0}
ubnd = CoefficientFunction([ubnd_dir[key] for key in mesh.GetBoundaries()])

As initial values we prescribe $u_0(x,y) = \sin(2\pi x)$:

In [ ]:
u0 = sin(2*pi*x)

## Discretization

### The mesh
For the mesh we take a 100x1 mesh (as the solution is constant in y-direction anyway). The time step is manually adjusted to provide stability in the sense of a CFL condition:

In [ ]:
N=100
mesh = MakeStructured2DMesh(nx=N,ny=1,quads=True,periodic_x=True,periodic_y=True)
dt=1/N

### A discretization with a central flux

In [ ]:
def fhatn_central(u1,u2,n):
    return 0.5*F(u1)*n+0.5*F(u2)*n # central flux
Solve(F,fhatn_central, u0, ubnd, mesh, dt)

**TODO:** 
1. 
Write a flux function `fhatn_central(u1,u2,n)` as an upwind flux:
$$
\hat{f}_{K,n}(u_1,u_2,n) = \left\{ \begin{array}{ccc} u_1 & \text{if} &\frac{\partial F}{\partial u}(u_1) \cdot n > 0 \\ u_2 & \text{else.} & \end{array} \right.
$$
2. 
Test your implementation. Is the discretization stable and or conservative? 

### A discretization with an upwind-type flux

**TODO:** 
1. 
Write a flux function `fhatn_central(u1,u2,n)` as an upwind flux:
$$
\hat{f}_{K,n}(u_1,u_2,n) = \left\{ \begin{array}{ccc} u_1 & \text{if} &\frac{\partial F}{\partial u}(u_1) \cdot n > 0 \\ u_2 & \text{else.} & \end{array} \right.
$$
2. 
Test your implementation. Is the discretization stable and or conservative? 

### A discretization with a Lax-Friedrichs flux

**TODO:** 
1. 
Write a flux function `fhatn_LF(u1,u2,n)` as a Lax-Friedrichs flux:
$$
\hat{f}_{K,n}(u_1,u_2,n) = \frac{F(u_1)+F(u_2)}{2} \cdot n+ |F_{\max}| \frac{u_1-u_2}{2}
$$
where $|F_{\max}| = \max_{v \in \{u_1,u_2\}} |\frac{\partial F}{\partial u}(v)|$.
2. 
Test your implementation. Is the discretization stable and or conservative? 